Load fast.ai computer vision libraries

In [2]:
from fastai.vision import *

<br/>

# Export Dataset

Create dataset image link files with following code:
```javascript
urls = Array.from(document.querySelectorAll('.rg_di .rg_meta')).map(el=>JSON.parse(el.textContent).ou);
window.open('data:text/csv;charset=utf-8,' + escape(urls.join('\n')));
```
and upload files to the server.

<br/>

Set folder and file names:

In [ ]:
folder = 'lobster'
file = 'urls_lobster'

In [ ]:
folder = 'scorpion'
file = 'urls_scorpion'

<br/>

Create folders in correct path:

In [ ]:
path = Path('data/lobsterscorpion')
dest = path/folder
dest.mkdir(parents=True, exist_ok=True)

<br/>

Download images in selected folder from selected file

In [ ]:
download_images(path/file, dest, max_pics=200)

<br/>

Check path for correct contents

In [ ]:
path.ls()

<br/>

Create classes array with folder naming

In [ ]:
classes = ['lobster','scorpion']

<br/>

Remove images that cannot be opened for any reason

In [ ]:
for c in classes:
        print(c)
        verify_images(path/c, delete=_True, max_size=500)

<br/>

# Finalize Data

Set random seed to get consistent results

In [ ]:
np.random.seed(42)

<br/>

Create databunch variable, normalizing all images and allocating 20% of them to a validaiton set

In [ ]:
data = ImageDataBunch.from_folder(path, train=".", valid_pct=0.2,
        ds_tfms=get_transforms(), size=224, num_workers=4).normalize(imagenet_stats)

<br/>

See a batch of iamges form the databunch to make sure all is well

In [ ]:
data.show_batch(rows=3, figsize=(7,8))

<br/>

Check category names, the number of categories, the size of the training set, and the size of the validaiton set

In [ ]:
data.classes, data.c, len(data.train_ds), len(data.valid_ds)

<br/>

# Train Model

Create Convolutional Neural Net(CNN) ResNet-34 deep learning model

In [ ]:
learn = cnn_learner(data, models.resnet34, metrics=error_rate)

Train model for 4 epoch cycles

In [ ]:
learn.fit_one_cycle(4)

Save model thus far

In [ ]:
learn.save('stage-1')

<br/>

Unfreeze all of the 34 layers (**even deeper learning.. heheh**)

In [ ]:
learn.unfreeze()

<br/>

Record learning rate throughout layers

In [ ]:
learn.lr_find()

<br/>

Plot learning rate

In [ ]:
learn.recorder.plot()

<br/>

Target the **slice** of the longest down-trending slope

In [ ]:
learn.fit_one_cycle(2, max_lr=slice(4e-6,7e-3))

<br/>

More training if necesary

In [ ]:
learn.fit_one_cycle(4)

<br/>

Save more accurate, deeper model

In [ ]:
learn.save('stage-2')

<br/>

# Interpret Results

Create interpretation variable

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

<br/>

Plot interpretation on confusion matrix

In [ ]:
interp.plot_confusion_matrix()

<br/>

Print file names for top losses

In [ ]:
losses,idxs = interp.top_losses(10)
for p in data.valid_ds.x.items[idxs]:
    print(p)

<br/>

# Export Model

In [ ]:
learn.export()